In [1]:
import os, cv2, random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

In [2]:
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from keras.models import Sequential
from keras.layers import Dropout, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D, Dense, Activation
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [3]:
TRAIN_DIR = 'F:/fisheries/train/train/'
TEST_DIR = 'F:/fisheries/test_stg1/test_stg1/'
FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
ROWS = 720  #720
COLS = 1280 #1280
CHANNELS = 3

In [4]:
def get_images(fish):
    """Load files from train folder"""
    fish_dir = TRAIN_DIR+'{}'.format(fish)
    images = [fish+'/'+im for im in os.listdir(fish_dir)]
    return images

def read_image(src):
    """Read and resize individual images"""
    im = cv2.imread(src, cv2.IMREAD_COLOR)
    im = cv2.resize(im, (COLS, ROWS), interpolation=cv2.INTER_CUBIC)
    return im


files = []
y_all = []

for fish in FISH_CLASSES:
    fish_files = get_images(fish)
    files.extend(fish_files)
    
    y_fish = np.tile(fish, len(fish_files))
    y_all.extend(y_fish)
    print("{0} photos of {1}".format(len(fish_files), fish))
    
y_all = np.array(y_all)

1719 photos of ALB
200 photos of BET
117 photos of DOL
67 photos of LAG
465 photos of NoF
299 photos of OTHER
176 photos of SHARK
734 photos of YFT


In [5]:
X_all = np.ndarray((len(files), ROWS, COLS, CHANNELS), dtype=np.uint8)

for i, im in enumerate(files): 
    X_all[i] = read_image(TRAIN_DIR+im)
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))

print(X_all.shape)

Processed 0 of 3777
Processed 1000 of 3777
Processed 2000 of 3777
Processed 3000 of 3777
(3777, 720, 1280, 3)


In [6]:
# One Hot Encoding Labels
y_all = LabelEncoder().fit_transform(y_all)
y_all = np_utils.to_categorical(y_all)

X_train, X_valid, y_train, y_valid = train_test_split(X_all, y_all, 
                                                    test_size=0.2, random_state=23, 
                                                    stratify=y_all)

In [8]:
optimizer = RMSprop(lr=1e-4)
objective = 'categorical_crossentropy'

def center_normalize(x):
    return (x - K.mean(x)) / K.std(x)

model = Sequential()

model.add(Activation(activation=center_normalize, input_shape=(ROWS, COLS, CHANNELS)))

model.add(Convolution2D(32, 5, 5, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(Convolution2D(32, 5, 5, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='tf'))

model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='tf'))

model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='tf'))

model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu', dim_ordering='tf'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='tf'))


model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(len(FISH_CLASSES)))
model.add(Activation('sigmoid'))

model.compile(loss=objective, optimizer=optimizer)

E:\Anaconda\envs\tensorflow2\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), padding="same", data_format="channels_last", activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
E:\Anaconda\envs\tensorflow2\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), padding="same", data_format="channels_last", activation="relu")`
  if sys.path[0] == '':
E:\Anaconda\envs\tensorflow2\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_last")`
  del sys.path[0]
E:\Anaconda\envs\tensorflow2\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", data_format="channels_last", activation="relu")`
  from ipykernel import kernelapp as app
E:\Anaco

In [12]:
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='auto')        
        
model.fit(X_train, y_train, batch_size=8, nb_epoch=20,
              validation_split=0.2, verbose=1, shuffle=True, callbacks=[early_stopping])

E:\Anaconda\envs\tensorflow2\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 2416 samples, validate on 605 samples
Epoch 1/20
2416/2416 [==============================] - 264s 109ms/step - loss: 1.7459 - val_loss: 1.4886
Epoch 2/20
2416/2416 [==============================] - 259s 107ms/step - loss: 1.5462 - val_loss: 1.2403
Epoch 3/20
2416/2416 [==============================] - 258s 107ms/step - loss: 1.3665 - val_loss: 0.9702
Epoch 4/20
2416/2416 [==============================] - 258s 107ms/step - loss: 1.1938 - val_loss: 0.8169
Epoch 5/20
2416/2416 [==============================] - 258s 107ms/step - loss: 1.0479 - val_loss: 0.7356
Epoch 6/20
2416/2416 [==============================] - 258s 107ms/step - loss: 0.9270 - val_loss: 0.5892
Epoch 7/20
2416/2416 [==============================] - 257s 107ms/step - loss: 0.7869 - val_loss: 0.4987
Epoch 8/20
2416/2416 [==============================] - 257s 106ms/step - loss: 0.7158 - val_loss: 0.4985
Epoch 9/20
2416/2416 [==============================] - 256s 106ms/step - loss: 0.6204 - val_loss: 0.4987

In [14]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='weights.hdf5', verbose=1, save_best_only=True)

model.fit(X_train, y_train, batch_size=8, nb_epoch=20,
              validation_split=0.2, verbose=1, shuffle=True, callbacks=[early_stopping,checkpointer])

E:\Anaconda\envs\tensorflow2\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Train on 2416 samples, validate on 605 samples
Epoch 1/20
2416/2416 [==============================] - 256s 106ms/step - loss: 0.3608 - val_loss: 0.5479

Epoch 00001: val_loss improved from inf to 0.54788, saving model to weights.hdf5
Epoch 2/20
2416/2416 [==============================] - 258s 107ms/step - loss: 0.3452 - val_loss: 0.3484

Epoch 00002: val_loss improved from 0.54788 to 0.34836, saving model to weights.hdf5
Epoch 3/20
2416/2416 [==============================] - 257s 106ms/step - loss: 0.3755 - val_loss: 0.4708

Epoch 00003: val_loss did not improve from 0.34836
Epoch 4/20
2416/2416 [==============================] - 256s 106ms/step - loss: 0.3736 - val_loss: 0.3959

Epoch 00004: val_loss did not improve from 0.34836
Epoch 5/20
2416/2416 [==============================] - 256s 106ms/step - loss: 0.3872 - val_loss: 0.7410

Epoch 00005: val_loss did not improve from 0.34836
Epoch 6/20
2416/2416 [==============================] - 256s 106ms/step - loss: 0.3878 - val_loss: 

In [11]:
model.save("model.h5")

In [12]:
preds = model.predict(X_valid, verbose=1)
print("Validation Log Loss: {}".format(log_loss(y_valid, preds)))

756/756 [==============================] - 2s 3ms/step
Validation Log Loss: 0.49139945581161554


In [13]:
test_files = [im for im in os.listdir(TEST_DIR)]
test = np.ndarray((len(test_files), ROWS, COLS, CHANNELS), dtype=np.uint8)

for i, im in enumerate(test_files): 
    test[i] = read_image(TEST_DIR+im)
    
test_preds= model.predict(test, verbose=1)

1000/1000 [==============================] - 2s 2ms/step


In [14]:
submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
submission.insert(0, 'image', test_files)
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,0.000002,0.000007,1.686936e-05,3.351741e-08,0.999840,2.218775e-07,1.460753e-07,1.071617e-07
1,img_00007.jpg,0.834539,0.000490,4.392459e-05,9.617465e-04,0.000185,2.875698e-05,5.457621e-05,3.838130e-03
2,img_00009.jpg,0.988598,0.018888,6.868071e-06,4.145420e-06,0.001383,3.315496e-02,1.448603e-03,5.484286e-05
3,img_00018.jpg,0.999667,0.000020,1.323739e-07,3.765064e-09,0.000329,4.232056e-04,5.893771e-07,8.607932e-06
4,img_00027.jpg,0.001283,0.017110,1.555771e-03,5.815837e-04,0.062930,1.540279e-01,2.252480e-02,1.248490e-02


In [15]:
submission.to_csv("submission.csv")

In [16]:
TEST_DIR2 = 'F:/fisheries/test_stg2/'
test_files2 = [im for im in os.listdir(TEST_DIR2)]
test2 = np.ndarray((len(test_files2), ROWS, COLS, CHANNELS), dtype=np.uint8)

for i, im in enumerate(test_files2): 
    test2[i] = read_image(TEST_DIR2+im)
    
test_preds2= model.predict(test2, verbose=1)

12153/12153 [==============================] - 22s 2ms/step


In [17]:
submission2 = pd.DataFrame(test_preds2, columns=FISH_CLASSES)
submission2.insert(0, 'image', test_files2)
submission2.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,image_00001.jpg,0.013289,0.004762,1.344359e-03,3.379114e-03,0.126330,1.902095e-03,1.792851e-04,0.012287
1,image_00002.jpg,0.184383,0.000001,9.613483e-07,6.177684e-08,0.113548,8.451569e-07,9.031543e-08,0.000005
2,image_00003.jpg,0.011997,0.000185,4.711205e-05,9.387301e-06,0.692124,1.240539e-02,7.988975e-06,0.000237
3,image_00004.jpg,0.997744,0.000012,9.240361e-08,7.515166e-09,0.000007,1.249844e-07,5.788507e-08,0.001703
4,image_00005.jpg,0.000170,0.000117,4.799098e-05,9.214643e-06,0.487955,6.838353e-06,7.121790e-07,0.003013


In [18]:
submission2.to_csv("submission2.csv")

In [24]:
TEST_DIR_all = 'F:/fisheries/test_all/'
test_files_all = [im for im in os.listdir(TEST_DIR_all)]
test_all = np.ndarray((len(test_files_all), ROWS, COLS, CHANNELS), dtype=np.uint8)

for i, im in enumerate(test_files_all): 
    test_all[i] = read_image(TEST_DIR_all+im)
    
test_preds_all= model.predict(test_all, verbose=1)

13153/13153 [==============================] - 23s 2ms/step


In [23]:
submission_all = pd.DataFrame(test_preds_all, columns=FISH_CLASSES)
submission_all.insert(0, 'image', test_files_all)
submission_all.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,image_00001.jpg,0.013289,0.004762,1.344354e-03,3.379105e-03,0.126330,1.902093e-03,1.792851e-04,0.012287
1,image_00002.jpg,0.184382,0.000001,9.613428e-07,6.177660e-08,0.113548,8.451505e-07,9.031509e-08,0.000005
2,image_00003.jpg,0.011997,0.000185,4.711197e-05,9.387328e-06,0.692124,1.240541e-02,7.988967e-06,0.000237
3,image_00004.jpg,0.997744,0.000012,9.240308e-08,7.515109e-09,0.000007,1.249844e-07,5.788473e-08,0.001703
4,image_00005.jpg,0.000170,0.000117,4.799080e-05,9.214581e-06,0.487956,6.838328e-06,7.121756e-07,0.003013


In [25]:
submission_all.to_csv("submission_all.csv")

In [34]:
TEST_DIR_STG1 = 'F:/fisheries/test_stg1/test_stg1/'
test_files_stg1 = [im for im in os.listdir(TEST_DIR_STG1)]
test_files = [im for im in os.listdir(TEST_DIR_STG1)]

TEST_DIR_STG2 = 'F:/fisheries/test_stg2/'
test_files_stg2 = [im for im in os.listdir(TEST_DIR_STG2)]
test_files = test_files + [ 'test_stg2/'+im for im in os.listdir(TEST_DIR_STG2)]

test = np.ndarray((len(test_files), ROWS, COLS, CHANNELS), dtype=np.uint8)

In [39]:
test_files[8000]

'test_stg2/image_07001.jpg'

In [31]:
len(test_files)

13153

In [35]:
i=0
for im in test_files_stg1:
    test[i] = read_image(TEST_DIR_STG1+im)
    i=i+1

for im in test_files_stg2:
    test[i] = read_image(TEST_DIR_STG2+im)
    i=i+1

In [36]:
test_preds= model.predict(test, verbose=1)

13153/13153 [==============================] - 23s 2ms/step


In [42]:
submission_all = pd.DataFrame(test_preds, columns=FISH_CLASSES)
submission_all.insert(0, 'image', test_files)
submission_all

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,2.128506e-06,7.054294e-06,1.686963e-05,3.351780e-08,9.998397e-01,2.218805e-07,1.460769e-07,1.071633e-07
1,img_00007.jpg,8.345395e-01,4.902654e-04,4.392463e-05,9.617392e-04,1.851140e-04,2.875709e-05,5.457647e-05,3.838106e-03
2,img_00009.jpg,9.885976e-01,1.888769e-02,6.868111e-06,4.145452e-06,1.383067e-03,3.315519e-02,1.448612e-03,5.484312e-05
3,img_00018.jpg,9.996670e-01,2.019401e-05,1.323717e-07,3.765014e-09,3.291467e-04,4.232054e-04,5.893714e-07,8.607849e-06
4,img_00027.jpg,1.283254e-03,1.711014e-02,1.555773e-03,5.815846e-04,6.293026e-02,1.540288e-01,2.252497e-02,1.248480e-02
5,img_00030.jpg,7.944880e-01,4.423991e-05,1.794508e-06,1.027486e-07,5.456215e-02,6.475948e-07,3.802587e-06,1.202150e-06
6,img_00040.jpg,4.669057e-03,1.536230e-01,2.073474e-04,4.317465e-04,3.661947e-03,1.298031e-02,4.209146e-02,2.897749e-03
7,img_00046.jpg,9.998835e-01,1.268638e-06,4.410723e-09,2.566909e-09,1.113256e-05,9.568805e-08,5.369383e-09,6.659988e-07
8,img_00053.jpg,9.999969e-01,9.955902e-07,4.499885e-10,4.745679e-10,3.730323e-06,6.506461e-10,1.534237e-09,1.174769e-08
9,img_00071.jpg,1.718050e-03,1.626486e-06,1.575531e-05,4.345075e-04,3.466967e-03,5.431809e-06,3.123965e-07,3.021032e-04


In [43]:
submission_all.to_csv("submission_all_2.csv")